**Check GPU connection**

In [ ]:
!nvidia-smi

**Connect to Google Drive**

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

**Import all required libraries**

In [ ]:
import random
import time
import tensorflow
import matplotlib.pyplot as plt
import keras.wrappers.scikit_learn
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop, Rescaling, Resizing
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from utilities import list_paths, create_dataset_and_labels

**Create folder for accuracy/loss charts and model summary**

In [ ]:
print("Input training name:")
training_name = input()
try:
    path = f"gdrive/MyDrive/BSL/models/{training_name}"
    !mkdir {path}
except:
    print('Path cannot be created')

**Convolutional Neural Network (CNN) with 4 conv layers**

In [ ]:
def list_paths(path):
    '''
    Lists all image paths and labels from a given directory.
    '''

    images = list()
    labels = list()
    for name in os.listdir(path):
        if name != ".DS_Store":
            labels.append(name)

    for label in labels:
        full_path = os.path.join(path, label)
        for img in os.listdir(full_path):
            images.append(os.path.join(full_path, img))

    return images


def create_dataset_and_labels(images_paths):
    ''' 
    Reads given paths and saves images/labels and puts them into
    separate lists
    '''

    images = list()
    labels = list()

    for path in images_paths:
        try:
            image = cv.imread(path)
            images.append(image)
            labels.append(path.split(os.path.sep)[-2])

        except:
            print(path + ' not found')
            continue

    images = np.array(images, dtype="float") / 255.0
    labels = np.array(labels)
    return images, labels


def create_accuracy_chart(history, name):
    ''' Creates Accuracy chart '''

    fig = plt.figure()
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    plt.plot(accuracy)
    plt.plot(val_accuracy)
    plt.title("ACCURACY CHART")
    plt.xlabel('Accuracy')
    plt.ylabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.savefig('gdrive/MyDrive/BSL/models/{}/Accuracy'.format(name))


def create_loss_chart(history, name):
    ''' Creates Loss chart '''

    fig = plt.figure()
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.plot(loss)
    plt.plot(val_loss)
    plt.title("LOSS CHART")
    plt.xlabel('Loss')
    plt.ylabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.savefig('gdrive/MyDrive/BSL/models/{}/Loss'.format(name))


def train(name):
    ''' Train model function '''

    input_shape = (300, 300, 3)

    path = "../trainingData"
    model_path = "./model"
    images_paths = list_paths(path)
    dataset, labels = create_dataset_and_labels(images_paths)

    # Gets training data and create keras train-dataset and test-dataset
    (train_x, test_x, train_y, test_y) = train_test_split(
        dataset,
        labels,
        test_size=0.25,
        random_state=42
    )

    encoder = LabelEncoder()
    train_y = encoder.fit_transform(train_y)
    test_y = encoder.fit_transform(test_y)

    # Data augmentation flips randomly input and rotates it by 0.3
    data_augmentation = keras.Sequential([
        layers.experimental.preprocessing.RandomFlip(
            "horizontal_and_vertical"),
        layers.experimental.preprocessing.RandomRotation(0.2),
    ])

    # Resizes data to (180, 180) and rescales it to be between 0 and 1 
    data_resize = keras.Sequential([
        Resizing(180, 180)
        # Rescaling(1./255)
    ])


    # Begining of the model
    # Input layer reshapes input to be compatibile

    # Augmentation resizing and rescaling
    x = data_resize(inputs)
    x = data_augmentation(x)

    # Convolutional layer 1
    x = layers.Conv2D(128, (3, 3), padding='same', activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional layer 2
    x = layers.Conv2D(128, (3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional layer 3
    x = layers.Conv2D(64, (3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional layer 4
    x = layers.Conv2D(64, (3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Dropout to prevent overfitting
    x = layers.Dropout(0.5)(x)

    # Flatten (batch, num)
    x = layers.Flatten()(x)

    # Dense layer with relu activation
    x = layers.Dense(512, activation="relu")(x)

    # Output layer
    outputs = layers.Dense(4, activation="softmax")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)

    # Creates and saves model summary as an image
    plot_model(
        model,
        to_file='./models/{}/model_summary.png'.format(name),
        show_layer_names=True,
        show_shapes=True
    )

    # Displays model summary in terminal
    model.summary()

    # Sets loss and optomazer algoriths
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Trains the model and saves it do history variable
    history = model.fit(train_x, train_y, batch_size=32,
                        epochs=32, validation_data=(test_x, test_y))

    # Create loss and accuracy functions
    create_accuracy_chart(history, name)
    create_loss_chart(history, name)

    # Saves model
    keras.models.save_model(
        model,
        model_path,
        overwrite=True,
        include_optimizer=True,
        save_format=None,
        signatures=None,
        options=None,
    )


train(training_name)